#### Import necessary packages

In [1]:
import pandas as pd
import numpy as np

## Read each participant's CSV file to their own dataframe

In [2]:
p1 = pd.read_csv('participant_1.csv')
p2 = pd.read_csv('participant_2.csv')
p3 = pd.read_csv('participant_4.csv')

### Show that participant_2.csv and participant_3.csv are the same file

In [3]:
pduplicate = pd.read_csv('participant_3.csv')

In [4]:
p2.equals(pduplicate)

True

## Merge the participant dataframes into a single dataframe

In [5]:
frames = [p1, p2, p3]

In [6]:
participants = pd.concat(frames, keys=['Participant 1', 'Participant 2', 'Participant 3'])

### Check shape of dataframe to see the number of datapoints before cleaning

In [7]:
participants.shape

(94373, 13)

## Remove null values and missing data

In [8]:
ptemp = participants.replace(0,np.nan)

In [9]:
ptemp.dropna(subset=['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'], inplace=True)

### Fix timestamp at 0 seconds

In [10]:
ptemp.loc['Participant 1'].at[0,'timestamp']=0

In [11]:
ptemp.loc['Participant 2'].at[0,'timestamp']=0

In [12]:
ptemp.loc['Participant 3'].at[0,'timestamp']=0

### Check shape of files post-cleaning to make sure there are still enough datapoints

In [13]:
ptemp.shape

(88507, 13)

Only ~6000 datapoints (~6%) were lost from cleaning; probably worth seeing how many more datapoints are lost due to a threshhold for left/right eye confidence

## Insert a confidence threshold

In [14]:
p_conf_r = ptemp[ptemp['r_conf'] > 0.25]

In [15]:
p_corr = p_conf_r[p_conf_r['l_conf'] > 0.25]

In [16]:
p_corr.shape

(83810, 13)

Requiring each eye to have a minimum confidence value of 0.2 removes a mere 4000 datapoints from the original 95000, yet likely confers a benefit from removing outliers.

I also noticed some of the pupil sizes were odd (a few were the value of Pi for example) so I will create another threshhold.

## Insert a pupil size threshold

In [17]:
p_pupil = p_corr[p_corr['r_size'] > 5]

In [18]:
p_final = p_pupil[p_pupil['l_size'] > 5]

In [19]:
p_final.shape

(83810, 13)

It seems the previous threshhold for confidence was enough to take care of pupil size outliers.

## TODO: Examine how blinking affects data

It may be worthwhile to see how the software handles and records blinking -- is blinking the cause of some of the missing/skewed data? If so, is blinking another piece of data that should be tracked?

### Reset index (messed up due to cleaning)

In [20]:
a1 = p_final.loc['Participant 1'].reset_index(drop=True)
a2 = p_final.loc['Participant 2'].reset_index(drop=True)
a3 = p_final.loc['Participant 3'].reset_index(drop=True)
fixed_index = pd.concat([a1, a2, a3], keys=['Participant 1','Participant 2','Participant 3'])

## Perform basic data analysis

### Group by block number and type

In [21]:
p1_blocknum = fixed_index.loc['Participant 1'].groupby(['block_number', 'block_type'], as_index=False)

In [22]:
p2_blocknum = fixed_index.loc['Participant 2'].groupby(['block_number', 'block_type'], as_index=False)

### Preliminary data analysis for participants 1 and 2

In [23]:
p1_analysis = p1_blocknum['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'].describe()

In [24]:
p2_analysis = p2_blocknum['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'].describe()

### Create column to track block number for statistical analysis

In [42]:
p1_analysis['block_number']=range(len(p1_analysis))
p1_analysis = p1_analysis[['block_number','r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos']]
p1_analysis

block_number r_size                                                        \
                 count        mean        std      min        25%        50%   
0             0  200.0   75.119788   1.534451  72.5879  73.939775   74.63810   
1             1  432.0   81.517885  10.616744  30.7274  75.523350   81.96940   
2             2  625.0   77.298602   2.750458  56.4005  75.035300   77.50660   
3             3   84.0  109.344910  66.105883  74.6420  76.076975   78.10700   
4             4  188.0   78.865911   2.201778  74.7267  77.550250   78.63180   
5             5  357.0   78.568041   3.421106  40.1633  77.047800   78.59410   
6             6  233.0   74.736615   1.796160  70.5051  73.893700   74.76510   
7             7  346.0   81.716425  13.622830  27.7907  77.546775   81.66275   
8             8   57.0   80.431168   2.654904  75.4359  77.607000   81.52120   
9             9  129.0   78.921981   5.605579  72.7842  75.370900   78.76170   
10           10  219.0   78.292483   2.712160  72.9418  76.106000   78.64750   
11           11  376.0   74.725645   2.853876  51.4010  72.608625   75.17820   
12           12  222.0   71.557097   2.367572  65.7816  70.586225   72.05570   
13           13  389.0   77.249710   2.440658  58.3322  75.513100   76.98090   
14           14  296.0   85.260654  23.831953  32.4377  74.217925   78.33640   
15           15  193.0   76.799670   1.760255  61.3347  75.959900   76.76680   
16           16  451.0   82.071559   6.587969  43.8933  76.391900   80.77720   
17           17  447.0   83.955536   6.049879  73.4876  79.905400   82.61870   
18           18  185.0   75.944581   2.230909  72.1401  74.094500   75.65520   
19           19  366.0   83.762189  12.354398  67.2629  77.353125   80.47435   
20           20  282.0   87.167748   8.763637  68.1462  79.438925   86.56740   
21           21  221.0   74.332635   1.672470  68.6766  73.555200   74.31510   
22           22  420.0   75.179333   4.160575  68.0680  71.737525   75.02525   
23           23  401.0   76.436210   7.296741  67.3667  72.464000   76.81400   
24           24  238.0   74.409642   1.741419  70.0336  73.354075   74.77830   
25           25  344.0   82.505729  26.862381  70.7852  74.302350   76.23630   
26           26  337.0   78.041997   6.397455  36.2836  76.626900   78.20750   
27           27  192.0   79.986336   5.927628  70.4267  76.189550   78.63595   
28           28  330.0   77.439783   4.622362  71.4407  74.756575   76.53230   
29           29  391.0   76.525133   5.857811  72.0564  74.298800   76.07660   
..          ...    ...         ...        ...      ...        ...        ...   
62           62  548.0   74.872091   2.481969  68.8620  73.183550   74.65000   
63           63  232.0   72.471700   2.539169  65.6097  70.406900   71.85020   
64           64  462.0   73.651826   4.847016  66.9834  70.481750   72.54930   
65           65  260.0   76.404256   3.364465  70.5016  74.684650   75.85485   
66           66  238.0   76.367476   1.939443  72.1501  75.536525   76.30905   
67           67  457.0   74.362579   1.900712  65.5086  72.923300   74.39130   
68           68  175.0   74.145177   3.740778  54.6001  72.631850   74.66440   
69           69  231.0   70.795605   3.946278  62.2438  68.158450   70.60570   
70           70  395.0   74.489902   4.153869  45.1650  71.905950   74.26370   
71           71  209.0  162.757443  77.597535  69.7272  74.248600  223.55300   
72           72  168.0   80.491065  12.522954  67.1332  78.241300   79.05920   
73           73  322.0  161.220165  76.972197  66.3193  80.002175  228.40000   
74           74  602.0   77.652960   5.883655  68.0343  74.359850   77.35235   
75           75  209.0   76.866341  16.316969  71.0739  74.028000   74.71340   
76           76  406.0   77.325180   3.811664  72.6800  74.786750   76.30275   
77           77  272.0   79.884321   2.780948  50.1597  79.209175   80.40640   
78           78  238.0   74.254544   1.638263  71.1160  73.247800   74.04510   
79 

In [41]:
p2_analysis['block_number']=range(len(p2_analysis))
p2_analysis = p2_analysis[['block_number','r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos']]
p2_analysis

block_number r_size                                               \
                 count        mean        std       min         25%   
0             0  218.0  103.893111   5.005330   96.3320   98.897750   
1             1  465.0   96.261304  10.075218   81.8861   85.875900   
2             2  765.0  117.363801   8.495578   96.3905  112.856000   
3             3  232.0  102.217900   5.640954   54.5974   99.163350   
4             4  460.0  107.914500   6.755175   94.6174  103.240250   
5             5  516.0  120.093529  10.805415  108.0980  115.031000   
6             6  232.0  102.091268   8.236619   87.3244   94.313050   
7             7  469.0  105.127349   9.717891   74.8186   97.682900   
8             8  495.0  108.038330   7.517870   89.4264  101.022000   
9             9  231.0   97.434909   8.897610   82.1637   91.424650   
10           10  470.0  106.093923   9.339488   92.0977  100.732500   
11           11  516.0  112.171213  12.244693   31.7863  103.925250   
12           12  228.0   93.779242   5.088607   86.1784   88.148125   
13           13  468.0  104.283360   6.538192   90.3375   99.264100   
14           14  699.0  122.767775   7.520357  105.5950  119.011500   
15           15  227.0  107.031996   5.854537   65.7960  102.261500   
16           16  470.0  103.703084   4.587326   88.0600  100.287500   
17           17  697.0  118.717658  13.784135   34.0684  112.506000   
18           18  224.0   88.677578   7.601165   70.9496   81.029600   
19           19  476.0   96.932150   9.381542   75.6696   89.686500   
20           20  683.0  108.563252   4.916539   98.6193  104.442500   
21           21  232.0   88.527410   8.619794   75.2502   80.778725   
22           22  450.0  109.658407   8.510561   53.5505  102.462500   
23           23  695.0  106.501670   7.090325   75.9766  100.969000   
24           24  231.0   91.583291   3.640427   85.6577   88.710750   
25           25  462.0   98.672778  10.715426   63.1869   86.386150   
26           26  890.0  109.965309   6.934521   87.9099  105.326000   
27           27  218.0   94.155069   6.948120   68.8673   88.360875   
28           28  468.0   99.982239   4.547025   90.5489   97.963675   
29           29  684.0  112.663993   7.502302   98.6700  108.269750   
..          ...    ...         ...        ...       ...         ...   
62           62  599.0  116.184376   8.628554   53.3177  111.723000   
63           63  221.0  102.273042   6.409100   82.3825   95.922700   
64           64  472.0  102.005643   4.668742   89.6670   98.775000   
65           65  486.0  110.779377   5.443912   96.1029  108.233500   
66           66  219.0   97.030934   6.338199   47.7841   93.419850   
67           67  464.0   96.143357   6.556793   84.0490   91.291300   
68           68  432.0   98.708525   5.580869   87.6316   93.634925   
69           69  228.0   93.910490   5.383626   83.9068   87.946900   
70           70  468.0   92.142990   4.313597   83.1237   88.719650   
71           71  465.0   93.731964   7.012445   77.1428   92.032200   
72           72  218.0  103.113463  20.993418   47.3685   95.887500   
73           73  461.0  103.558648   3.403568   84.1364  101.700000   
74           74  553.0  110.342332   6.862384   97.4405  105.057000   
75           75  238.0  103.828075   4.432187   91.3258  101.018500   
76           76  446.0   91.766816   5.096200   60.3498   88.138725   
77           77  490.0   99.018887  30.520579   49.8837   87.616925   
78           78  225.0   85.368934   7.272810   76.9997   81.409700   
79           79  437.0   83.508964   5.904626   64.9972   77.866800   
80           80  515.0   95.658547   6.541407   80.1700   91.991900   
81           81  231.0   94.911455   6.007544   85.1225   89.159350   
82           82  467.0   82.579121   6.691238   73.0680   76.931800   
83           83  650.0  102.996926   9.180193   60.7863   98.201575   
84           84  233.0   96.133678   6.737013   87.6641   90.405000   
85           85  425.0   92

### Participant 3 has no data for block number or type, so I cannot group the eye data by question

In [27]:
p3_describe = fixed_index.loc['Participant 3'].describe()

#### Fix participant 3 dataframe to have same levels of index as participants 1 and 2 (there's likely a better way to do this)

In [28]:
interim = pd.concat([p3_describe, p3_describe], keys=['Participant 3'])

In [29]:
p3_analysis = pd.concat([interim, interim], keys=['Participant 3 statistics'], axis=1)
p3_analysis

Participant 3 statistics                              \
                                   timestamp        r_size        r_conf   
Participant 3 count             15423.000000  15423.000000  15423.000000   
              mean                285.559568   2852.633492      0.959265   
              std                 163.199200   4509.885676      0.066277   
              min                   0.000000    857.124210      0.264706   
              25%                 140.083000   1410.812084      0.956957   
              50%                 291.427000   1638.301183      0.970000   
              75%                 426.619000   1938.099744      1.000000   
              max                 569.619000  56062.410156      1.000000   

                                                                              \
                          r_x_pos       r_y_pos         l_size        l_conf   
Participant 3 count  15423.000000  15423.000000   15423.000000  15423.000000   
              mean     355.483903     72.265906   10908.206851      0.910927   
              std       47.894952     62.061069    6327.209585      0.042625   
              min      108.849000   -451.685000    1544.691925      0.257143   
              25%      318.419500     44.917850    9024.120915      0.904375   
              50%      358.046000     60.251600   10263.304281      0.913224   
              75%      387.100000    100.428000   11761.513862      0.920800   
              max     1269.800000   4273.340000  178922.711408      1.000000   

                                                                         
                          l_x_pos       l_y_pos block_number difficulty  
Participant 3 count  15423.000000  15423.000000          0.0        0.0  
              mean     355.483903     72.265906          NaN        NaN  
              std       47.894952     62.061069          NaN        NaN  
              min      108.849000   -451.685000          NaN        NaN  
              25%      318.419500     44.917850          NaN        NaN  
              50%      358.046000     60.251600          NaN        NaN  
              75%      387.100000    100.428000          NaN        NaN  
              max     1269.800000   4273.340000          NaN        NaN

### Brief analysis of participant 3

From a mere glance, it is obvious that there are outliers that I have missed; r_size and l_size have massive differences between 75th percentile and max values. I attempted to fix this by using a threshold for r_size and l_size, but it is difficult to do without losing a lot of data points. I could not find a nice balance between a (relatively) low standard deviation and a large dataset. Furthermore, I realized that the x and y position is the exact same value for each eye, leading me to believe that you tested a cyclops. However, the pupil size data is different, so you must have found the one cyclops with polycoria (thanks Google!)

I will not include this dataframe in the final exported CSV because I do not believe the data is useful enough to justify the time spent (it would take me a while to figure out how to make this play nicely with the other analyses grouped by block number).

## Merge the analysis dataframes

In [30]:
analysis = pd.concat([p1_analysis, p2_analysis], keys=['Participant 1','Participant 2'])
analysis

block_number r_size                                   \
                               count        mean        std       min   
Participant 1 0             0  200.0   75.119788   1.534451   72.5879   
              1             1  432.0   81.517885  10.616744   30.7274   
              2             2  625.0   77.298602   2.750458   56.4005   
              3             3   84.0  109.344910  66.105883   74.6420   
              4             4  188.0   78.865911   2.201778   74.7267   
              5             5  357.0   78.568041   3.421106   40.1633   
              6             6  233.0   74.736615   1.796160   70.5051   
              7             7  346.0   81.716425  13.622830   27.7907   
              8             8   57.0   80.431168   2.654904   75.4359   
              9             9  129.0   78.921981   5.605579   72.7842   
              10           10  219.0   78.292483   2.712160   72.9418   
              11           11  376.0   74.725645   2.853876   51.4010   
              12           12  222.0   71.557097   2.367572   65.7816   
              13           13  389.0   77.249710   2.440658   58.3322   
              14           14  296.0   85.260654  23.831953   32.4377   
              15           15  193.0   76.799670   1.760255   61.3347   
              16           16  451.0   82.071559   6.587969   43.8933   
              17           17  447.0   83.955536   6.049879   73.4876   
              18           18  185.0   75.944581   2.230909   72.1401   
              19           19  366.0   83.762189  12.354398   67.2629   
              20           20  282.0   87.167748   8.763637   68.1462   
              21           21  221.0   74.332635   1.672470   68.6766   
              22           22  420.0   75.179333   4.160575   68.0680   
              23           23  401.0   76.436210   7.296741   67.3667   
              24           24  238.0   74.409642   1.741419   70.0336   
              25           25  344.0   82.505729  26.862381   70.7852   
              26           26  337.0   78.041997   6.397455   36.2836   
              27           27  192.0   79.986336   5.927628   70.4267   
              28           28  330.0   77.439783   4.622362   71.4407   
              29           29  391.0   76.525133   5.857811   72.0564   
...                       ...    ...         ...        ...       ...   
Participant 2 62           62  599.0  116.184376   8.628554   53.3177   
              63           63  221.0  102.273042   6.409100   82.3825   
              64           64  472.0  102.005643   4.668742   89.6670   
              65           65  486.0  110.779377   5.443912   96.1029   
              66           66  219.0   97.030934   6.338199   47.7841   
              67           67  464.0   96.143357   6.556793   84.0490   
              68           68  432.0   98.708525   5.580869   87.6316   
              69           69  228.0   93.910490   5.383626   83.9068   
              70           70  468.0   92.142990   4.313597   83.1237   
              71           71  465.0   93.731964   7.012445   77.1428   
              72           72  218.0  103.113463  20.993418   47.3685   
              73           73  461.0  103.558648   3.403568   84.1364   
              74           74  553.0  110.342332   6.862384   97.4405   
              75           75  238.0  103.828075   4.432187   91.3258   
              76           76  446.0   91.766816   5.096200   60.3498   
              77           77  490.0   99.018887  30.520579   49.8837   
              78           78  225.0   85.368934   7.272810   76.9997   
              79           79  437.0   83.508964   5.904626   64.9972   
              80           80  515.0   95.658547   6.541407   80.1700   
              81           81  231.0   94.911455   6.007544   85.1225   
              82           82  467.0   82.579121   6.691238   73.0680   
              83           83  650.0  102.996926   9.180193   60.7863   
  

### Fix previously merged and cleaned data to have same levels of index (there's likely a better way to do this)

In [31]:
fixeddf = pd.concat([fixed_index, fixed_index], keys=['Experiment Data'], axis=1)
fixeddf

Experiment Data                                    \
                          timestamp        r_size    r_conf   r_x_pos   
Participant 1 0               0.000     80.530000  0.940600   302.805   
              1               0.116     82.526900  0.947200   302.112   
              2               0.132     83.706400  0.941765   302.712   
              3               0.168     83.795700  0.946122   254.151   
              4               0.201     84.952300  0.940600   263.454   
              5               0.216     84.053500  0.947200   302.817   
              6               0.337     84.478900  0.946122   301.612   
              7               0.385     84.623200  0.940600   301.170   
              8               0.400     84.140800  0.940600   300.214   
              9               0.452     83.843800  0.939388   328.365   
              10              0.468     83.822100  0.940600   304.063   
              11              0.484     83.543200  0.940600   304.700   
              12              0.604     83.712700  0.940600   305.609   
              13              0.652     83.522900  0.939388   305.214   
              14              0.672     83.241300  0.940600   304.890   
              15              0.704     82.708100  0.941765   346.798   
              16              0.720     82.871000  0.940600   304.678   
              17              0.804     82.180600  0.940600   304.748   
              18              0.840     82.235500  0.947200   308.966   
              19              0.956     81.308300  0.947200   309.772   
              20              1.040     80.103000  0.947200   192.357   
              21              1.092     78.860400  0.952857   309.197   
              22              1.156     78.317200  0.947200   309.649   
              23              1.192     78.554800  0.940600   309.653   
              24              1.260     77.349400  0.940600   300.890   
              25              1.344     76.998300  0.947200   300.517   
              26              1.460     77.145500  0.940600   343.589   
              27              1.476     77.330200  0.934000   341.771   
              28              1.560     77.322100  0.940600   305.607   
              29              1.828     74.957600  0.946122   303.058   
...                             ...           ...       ...       ...   
Participant 3 15393         563.351  15165.770922  0.472222   356.873   
              15394         563.387  13462.213394  0.676471   366.527   
              15395         563.419  12539.024379  0.735294   359.928   
              15396         564.895   9467.515413  0.823529   452.957   
              15397         565.063  30326.360630  0.500000   535.285   
              15398         565.265  10521.141006  0.794118  1269.800   
              15399         565.699  13512.604097  0.666667   361.884   
              15400         565.799  11986.539640  0.818182   373.457   
              15401         565.867  13528.720966  0.685714   352.578   
              15402         565.899  14069.035749  0.600000   359.760   
              15403         565.999  11391.575054  0.781250   371.055   
              15404         566.067  11354.677758  0.787879   371.589   
              15405         566.135  20437.033101  0.514286   353.865   
              15406         566.203  11836.041278  0.818182   365.027   
              15407         566.267  34918.338423  0.388889   379.824   
              15408         566.671   1701.618020  0.294118   322.182   
              15409         566.703  13590.391539  0.611111   320.476   
              15410         566.739  14139.257888  0.514286   335.910   
              15411         567.395  11243.593196  0.727273   370.571   
              15412         567.507  11699.488428  0.800000   354.237   
              15413         567.577   7256.245767  0.747431   419.327   
              15414         568.211  10568.427014  0.657143   126.417   
     

By examining the table, I can see that I missed a few outliers that are drastically affecting the data.

### Merge analyzed data with cleaned raw data

In [32]:
final = pd.merge(fixeddf, analysis, how='outer', left_index=True, right_index=True)
final

Experiment Data                                    \
                          timestamp        r_size    r_conf   r_x_pos   
Participant 1 0               0.000     80.530000  0.940600   302.805   
              1               0.116     82.526900  0.947200   302.112   
              2               0.132     83.706400  0.941765   302.712   
              3               0.168     83.795700  0.946122   254.151   
              4               0.201     84.952300  0.940600   263.454   
              5               0.216     84.053500  0.947200   302.817   
              6               0.337     84.478900  0.946122   301.612   
              7               0.385     84.623200  0.940600   301.170   
              8               0.400     84.140800  0.940600   300.214   
              9               0.452     83.843800  0.939388   328.365   
              10              0.468     83.822100  0.940600   304.063   
              11              0.484     83.543200  0.940600   304.700   
              12              0.604     83.712700  0.940600   305.609   
              13              0.652     83.522900  0.939388   305.214   
              14              0.672     83.241300  0.940600   304.890   
              15              0.704     82.708100  0.941765   346.798   
              16              0.720     82.871000  0.940600   304.678   
              17              0.804     82.180600  0.940600   304.748   
              18              0.840     82.235500  0.947200   308.966   
              19              0.956     81.308300  0.947200   309.772   
              20              1.040     80.103000  0.947200   192.357   
              21              1.092     78.860400  0.952857   309.197   
              22              1.156     78.317200  0.947200   309.649   
              23              1.192     78.554800  0.940600   309.653   
              24              1.260     77.349400  0.940600   300.890   
              25              1.344     76.998300  0.947200   300.517   
              26              1.460     77.145500  0.940600   343.589   
              27              1.476     77.330200  0.934000   341.771   
              28              1.560     77.322100  0.940600   305.607   
              29              1.828     74.957600  0.946122   303.058   
...                             ...           ...       ...       ...   
Participant 3 15393         563.351  15165.770922  0.472222   356.873   
              15394         563.387  13462.213394  0.676471   366.527   
              15395         563.419  12539.024379  0.735294   359.928   
              15396         564.895   9467.515413  0.823529   452.957   
              15397         565.063  30326.360630  0.500000   535.285   
              15398         565.265  10521.141006  0.794118  1269.800   
              15399         565.699  13512.604097  0.666667   361.884   
              15400         565.799  11986.539640  0.818182   373.457   
              15401         565.867  13528.720966  0.685714   352.578   
              15402         565.899  14069.035749  0.600000   359.760   
              15403         565.999  11391.575054  0.781250   371.055   
              15404         566.067  11354.677758  0.787879   371.589   
              15405         566.135  20437.033101  0.514286   353.865   
              15406         566.203  11836.041278  0.818182   365.027   
              15407         566.267  34918.338423  0.388889   379.824   
              15408         566.671   1701.618020  0.294118   322.182   
              15409         566.703  13590.391539  0.611111   320.476   
              15410         566.739  14139.257888  0.514286   335.910   
              15411         567.395  11243.593196  0.727273   370.571   
              15412         567.507  11699.488428  0.800000   354.237   
              15413         567.577   7256.245767  0.747431   419.327   
              15414         568.211  10568.427014  0.657143   126.417   
     

## Export to CSV

In [33]:
final.to_csv('C:\\Users\\Zach\\Desktop\\Senseye Challenge\\processed_data\\Processed Data.csv')

## Compare participants (keeping separate from exported dataframe)

### At the block_number level

#### Right eye size comparison

In [34]:
final.loc['Participant 1','r_size']

,count,mean,std,min,25%,50%,75%,max
0,200.0,75.119788,1.534451,72.5879,73.939775,74.63810,76.303475,79.0882
1,432.0,81.517885,10.616744,30.7274,75.523350,81.96940,84.437275,202.6180
2,625.0,77.298602,2.750458,56.4005,75.035300,77.50660,79.452400,85.1388
3,84.0,109.344910,66.105883,74.6420,76.076975,78.10700,84.619550,262.1780
4,188.0,78.865911,2.201778,74.7267,77.550250,78.63180,79.766425,89.5324
5,357.0,78.568041,3.421106,40.1633,77.047800,78.59410,80.154900,104.9320
6,233.0,74.736615,1.796160,70.5051,73.893700,74.76510,75.562600,82.4849
7,346.0,81.716425,13.622830,27.7907,77.546775,81.66275,84.507225,163.9620
8,57.0,80.431168,2.654904,75.4359,77.607000,81.52120,82.397600,83.7776
9,129.0,78.921981,5.605579,72.7842,75.370900,78.76170,80.246800,121.8560


In [35]:
final.loc['Participant 2','r_size']

,count,mean,std,min,25%,50%,75%,max
0,218.0,103.893111,5.005330,96.3320,98.897750,104.01650,108.22425,113.9510
1,465.0,96.261304,10.075218,81.8861,85.875900,96.15760,102.68200,117.5470
2,765.0,117.363801,8.495578,96.3905,112.856000,119.41600,122.62400,173.5030
3,232.0,102.217900,5.640954,54.5974,99.163350,101.38450,106.02750,112.1830
4,460.0,107.914500,6.755175,94.6174,103.240250,107.41550,111.23975,122.6710
5,516.0,120.093529,10.805415,108.0980,115.031000,118.95550,122.76600,250.7220
6,232.0,102.091268,8.236619,87.3244,94.313050,104.96150,109.85400,113.0760
7,469.0,105.127349,9.717891,74.8186,97.682900,102.96100,110.00100,142.4770
8,495.0,108.038330,7.517870,89.4264,101.022000,107.83300,113.23950,125.3280
9,231.0,97.434909,8.897610,82.1637,91.424650,97.06740,105.07550,113.6980


### At the question level (block_number in groups of three)

In [36]:
test = fixed_index.loc['Participant 1'].groupby(['block_number', 'block_type'])

In [37]:
p1_analysis_q = test['r_size','r_conf','r_x_pos','r_y_pos','l_size','l_conf','l_x_pos','l_y_pos'].describe()
p1_analysis_q

r_size                                             \
                          count        mean        std      min        25%   
block_number block_type                                                      
1.0          TRIAL_START  200.0   75.119788   1.534451  72.5879  73.939775   
2.0          BEGIN_SPAN   432.0   81.517885  10.616744  30.7274  75.523350   
3.0          CUE_ANSWER   625.0   77.298602   2.750458  56.4005  75.035300   
4.0          RESPONSE      84.0  109.344910  66.105883  74.6420  76.076975   
5.0          BEGIN_SPAN   188.0   78.865911   2.201778  74.7267  77.550250   
6.0          CUE_ANSWER   357.0   78.568041   3.421106  40.1633  77.047800   
7.0          RESPONSE     233.0   74.736615   1.796160  70.5051  73.893700   
8.0          BEGIN_SPAN   346.0   81.716425  13.622830  27.7907  77.546775   
9.0          CUE_ANSWER    57.0   80.431168   2.654904  75.4359  77.607000   
10.0         RESPONSE     129.0   78.921981   5.605579  72.7842  75.370900   
11.0         BEGIN_SPAN   219.0   78.292483   2.712160  72.9418  76.106000   
12.0         CUE_ANSWER   376.0   74.725645   2.853876  51.4010  72.608625   
13.0         RESPONSE     222.0   71.557097   2.367572  65.7816  70.586225   
14.0         BEGIN_SPAN   389.0   77.249710   2.440658  58.3322  75.513100   
15.0         CUE_ANSWER   296.0   85.260654  23.831953  32.4377  74.217925   
16.0         RESPONSE     193.0   76.799670   1.760255  61.3347  75.959900   
17.0         BEGIN_SPAN   451.0   82.071559   6.587969  43.8933  76.391900   
18.0         CUE_ANSWER   447.0   83.955536   6.049879  73.4876  79.905400   
19.0         RESPONSE     185.0   75.944581   2.230909  72.1401  74.094500   
20.0         BEGIN_SPAN   366.0   83.762189  12.354398  67.2629  77.353125   
21.0         CUE_ANSWER   282.0   87.167748   8.763637  68.1462  79.438925   
22.0         RESPONSE     221.0   74.332635   1.672470  68.6766  73.555200   
23.0         BEGIN_SPAN   420.0   75.179333   4.160575  68.0680  71.737525   
24.0         CUE_ANSWER   401.0   76.436210   7.296741  67.3667  72.464000   
25.0         RESPONSE     238.0   74.409642   1.741419  70.0336  73.354075   
26.0         BEGIN_SPAN   344.0   82.505729  26.862381  70.7852  74.302350   
27.0         CUE_ANSWER   337.0   78.041997   6.397455  36.2836  76.626900   
28.0         RESPONSE     192.0   79.986336   5.927628  70.4267  76.189550   
29.0         BEGIN_SPAN   330.0   77.439783   4.622362  71.4407  74.756575   
30.0         CUE_ANSWER   391.0   76.525133   5.857811  72.0564  74.298800   
...                         ...         ...        ...      ...        ...   
63.0         CUE_ANSWER   548.0   74.872091   2.481969  68.8620  73.183550   
64.0         RESPONSE     232.0   72.471700   2.539169  65.6097  70.406900   
65.0         BEGIN_SPAN   462.0   73.651826   4.847016  66.9834  70.481750   
66.0         CUE_ANSWER   260.0   76.404256   3.364465  70.5016  74.684650   
67.0         RESPONSE     238.0   76.367476   1.939443  72.1501  75.536525   
68.0         BEGIN_SPAN   457.0   74.362579   1.900712  65.5086  72.923300   
69.0         CUE_ANSWER   175.0   74.145177   3.740778  54.6001  72.631850   
70.0         RESPONSE     231.0   70.795605   3.946278  62.2438  68.158450   
71.0         BEGIN_SPAN   395.0   74.489902   4.153869  45.1650  71.905950   
72.0         CUE_ANSWER   209.0  162.757443  77.597535  69.7272  74.248600   
73.0         RESPONSE     168.0   80.491065  12.522954  67.1332  78.241300   
74.0         BEGIN_SPAN   322.0  161.220165  76.972197  66.3193  80.002175   
75.0         CUE_ANSWER   602.0   77.652960   5.883655  68.0343  74.359850   
76.0         RESPONSE     209.0   76.866341  16.316969  71.0739  74.028000   
77.0         BEGIN_SPAN   406.0   77.325180   3.811664  72.6800  74.786750   
78.0         CUE_ANSWER   272.0   79.884321   2.780948  50.1597  79.209175   
79.0         RESPONSE     238.0   74.254544   1.638263  71.1160  73.247800   
80.0         BEGIN_SPAN   467.0   77.279831  19

In [38]:
for i in test: print(i)

((1.0, 'TRIAL_START'),      timestamp   r_size    r_conf  r_x_pos  r_y_pos   l_size    l_conf  \
720     16.612  76.3407  0.934000  305.558  156.045  68.6006  0.970588   
721     16.632  75.5930  0.946122  306.926  156.791  65.7260  1.000000   
722     16.648  75.2591  0.940600  307.813  156.013  63.3464  0.972222   
723     16.664  76.0787  0.940600  304.206  154.546  71.0415  0.942857   
724     16.680  75.4883  0.946122  308.524  157.225  65.6531  1.000000   
725     16.696  74.7214  0.939388  305.181  154.344  70.3886  0.942857   
726     16.716  74.4932  0.940600  305.349  155.324  69.5650  0.972222   
727     16.732  74.6624  0.940600  305.964  157.696  67.2003  0.971429   
728     16.748  74.6028  0.940600  300.982  157.153  75.2925  0.944444   
729     16.764  75.2370  0.940600  307.312  155.529  66.8164  1.000000   
730     16.780  74.6233  0.939388  308.632  155.872  65.3055  0.972222   
731     16.800  74.9588  0.940600  306.453  154.765  65.6931  0.970588   
732     16.817 

[376 rows x 13 columns])
((13.0, 'RESPONSE'),       timestamp   r_size    r_conf  r_x_pos  r_y_pos   l_size    l_conf  \
3966     97.220  74.5635  0.940600  301.717  143.445  66.5926  0.970588   
3967     97.236  74.1009  0.934000  304.191  142.915  62.9966  0.972222   
3968     97.253  73.8711  0.939388  300.950  139.808  67.5579  0.944444   
3969     97.272  73.9555  0.940600  296.491  143.884  73.8234  0.916667   
3970     97.288  73.2545  0.940600  303.499  141.160  64.5593  0.944444   
3971     97.304  72.3915  0.939388  297.886  140.685  73.6328  0.857143   
3972     97.320  72.3574  0.940600  304.755  142.599  63.3816  0.942857   
3973     97.336  71.3892  0.946122  305.100  143.983  62.7489  1.000000   
3974     97.356  71.6683  0.940600  296.692  145.772  73.0890  0.941176   
3975     97.372  71.6321  0.940600  303.378  141.450  65.2186  0.942857   
3976     97.388  71.8754  0.940600  302.086  139.808  65.9733  0.941176   
3977     97.404  71.3869  0.940600  301.692  145.324  

((26.0, 'BEGIN_SPAN'),       timestamp    r_size    r_conf  r_x_pos   r_y_pos    l_size    l_conf  \
8077    185.336   73.9346  0.940600  299.170  144.3130   67.2757  0.972222   
8078    185.352   74.3453  0.934000  299.282  146.0580   66.8113  0.971429   
8079    185.369   73.9949  0.939388  299.217  142.0870   66.7641  0.971429   
8080    185.388   74.2902  0.934000  302.783  144.9470   63.2828  0.942857   
8081    185.404   73.6782  0.939388  301.256  146.6410   65.1752  0.972222   
8082    185.420   74.2965  0.934000  299.603  146.1580   66.5389  0.972222   
8083    185.436   73.4044  0.939388  300.651  144.4210   64.1233  0.941176   
8084    185.452   73.7660  0.934000  303.252  147.7180   62.2850  0.941176   
8085    185.472   73.3964  0.934000  298.139  148.7670   67.7166  1.000000   
8086    185.488   73.7468  0.940600  293.730  149.4420   70.0072  0.929802   
8087    185.505   73.3975  0.934000  288.141  152.2490   69.5030  0.926122   
8088    185.520   74.1145  0.934000  293.

[197 rows x 13 columns])
((41.0, 'BEGIN_SPAN'),        timestamp   r_size    r_conf  r_x_pos  r_y_pos   l_size    l_conf  \
12653    284.825  72.5586  0.934000  273.280  143.434  70.4684  0.944444   
12654    284.844  72.8735  0.934000  278.774  141.856  61.0766  0.971429   
12655    284.860  72.8441  0.934000  276.778  144.633  65.9208  0.970588   
12656    284.876  72.5957  0.934000  275.194  142.565  68.3512  0.942857   
12657    284.892  72.5677  0.934000  279.648  142.203  61.4535  1.000000   
12658    284.908  72.8812  0.934000  281.777  143.356  60.9777  0.944444   
12659    284.928  72.3873  0.934000  276.266  138.111  66.2197  0.861111   
12660    284.944  72.6722  0.927400  279.126  142.945  63.0155  1.000000   
12661    284.960  71.8385  0.932653  280.883  139.829  59.5290  1.000000   
12662    284.976  71.9705  0.934000  274.992  139.301  67.7265  0.942857   
12663    284.992  72.2507  0.934000  274.520  139.000  67.2441  0.910600   
12664    285.012  70.8759  0.932653  275

[287 rows x 13 columns])
((58.0, 'RESPONSE'),        timestamp    r_size    r_conf  r_x_pos   r_y_pos    l_size    l_conf  \
17760    385.272  199.1730  0.694444  300.348   41.4207   76.1786  0.899231   
17761    385.288  103.4620  0.777778  285.941   73.2998   73.1843  0.861111   
17762    385.308   93.5072  0.882353  292.490   88.2757   78.2269  0.888889   
17763    385.326   97.4312  0.857143  289.957   78.2961   79.7277  0.888889   
17764    385.356   79.3807  0.857143  304.125  115.4750   70.8463  0.944444   
17765    385.372  103.8670  0.696970  288.350   78.4371   72.6870  0.942857   
17766    385.392   94.1430  0.882353  294.606   89.8819   82.6079  0.914286   
17767    385.408  134.1270  0.823529  265.272   94.4300   71.8507  0.888889   
17768    385.424   74.5642  0.750000  291.520  121.8910   74.0979  0.916667   
17769    385.440   85.6515  1.000000  301.173  107.1830   72.0701  0.942857   
17770    385.456   85.5401  1.000000  298.552   97.5277   77.7390  0.878788   
17771 

((74.0, 'BEGIN_SPAN'),        timestamp   r_size    r_conf  r_x_pos  r_y_pos    l_size    l_conf  \
22590    474.180  79.1176  0.934000  291.372  270.025   76.8248  0.941511   
22591    474.196  78.6382  0.946122  291.540  269.438   76.0408  0.950956   
22592    474.212  79.7786  0.935294  292.268  269.159   76.0669  0.950400   
22593    474.228  79.3328  0.934000  292.259  270.388   75.8438  0.956882   
22594    474.244  79.8076  0.940600  291.078  269.802   77.3732  0.944992   
22595    474.264  80.3023  0.934000  291.482  270.473   76.9450  0.959592   
22596    474.280  79.9523  0.939388  292.947  268.984   75.2365  1.000000   
22597    474.296  79.2517  0.940600  290.717  266.812   76.9151  0.935817   
22598    474.312  79.6338  0.940600  290.593  265.972   76.4413  0.931257   
22599    474.329  80.0945  0.947200  289.656  264.717   73.4374  1.000000   
22600    474.348  79.2515  0.952857  290.732  263.892   76.3013  0.961923   
22601    474.364  79.7783  1.000000  283.497  274.266